In [72]:
import numpy as np
import pandas as pd

In [73]:
# load anime dataset
anime_df = pd.read_csv('dataset/anime.csv')
usecols = ["MAL_ID", "Name", "Score", "Genres", "Type", "Episodes", "Premiered",
           "Studios", "Source", "Rating", "Members", "Completed"]

anime_df=pd.read_csv('dataset/anime.csv',usecols=usecols)
anime_df.head()

,MAL_ID,Name,Score,Genres,Type,Episodes,Premiered,Studios,Source,Rating,Members,Completed
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,Spring 1998,Sunrise,Original,R - 17+ (violence & profanity),1251960,718161
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Movie,1,Unknown,Bones,Original,R - 17+ (violence & profanity),273145,208333
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",TV,26,Spring 1998,Madhouse,Manga,PG-13 - Teens 13 or older,558913,343492
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, Magic",TV,26,Summer 2002,Sunrise,Original,PG-13 - Teens 13 or older,94683,46165
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",TV,52,Fall 2004,Toei Animation,Manga,PG - Children,13224,7314


In [147]:
# load top users with anime rating dataset
rating_popular_anime = pd.read_csv('dataset/top_anime_unsupervised_use.csv')
rating_data = rating_popular_anime.merge(anime_df, left_on = 'MAL_ID', right_on = 'MAL_ID', how = 'left')
rating_data = rating_data[["user_id", "Name", "MAL_ID","rating"]]
combine_anime_rating = rating_data.dropna(axis = 0, subset = ['Name'])
anime_ratingCount = (combine_anime_rating.groupby(by = ['Name'])['rating'].count().reset_index()[['Name', 'rating']])
rating_data = combine_anime_rating.merge(anime_ratingCount, left_on = 'Name', right_on = 'Name', how = 'left')
rating_data = rating_data.drop(columns = "rating_x")
rating_data = rating_data.rename(columns={"rating_y": "rating"})
rating_data.head()

,user_id,Name,MAL_ID,rating
0,478,Witch Hunter Robin,7,853
1,478,Eyeshield 21,15,864
2,478,Hachimitsu to Clover,16,1374
3,478,Naruto,20,2211
4,478,Tennis no Ouji-sama,22,821


In [148]:
# # Encoding categorical data
# user_ids = rating_data["user_id"].unique().tolist()
# user2user_encoded = {x: i for i, x in enumerate(user_ids)}
# user_encoded2user = {i: x for i, x in enumerate(user_ids)}
# rating_data["user"] = rating_data["user_id"].map(user2user_encoded)
# n_users = len(user2user_encoded)
#
# anime_ids = rating_data["MAL_ID"].unique().tolist()
# anime2anime_encoded = {x: i for i, x in enumerate(anime_ids)}
# anime_encoded2anime = {i: x for i, x in enumerate(anime_ids)}
# rating_data["anime"] = rating_data["MAL_ID"].map(anime2anime_encoded)
# n_animes = len(anime2anime_encoded)
#
# print("Num of users: {}, Num of animes: {}".format(n_users, n_animes))
# print("Min total rating: {}, Max total rating: {}".format(min(rating_data['rating']), max(rating_data['rating'])))

In [151]:
g = rating_data.groupby('user_id')['rating'].count()
top_users = g.dropna().sort_values(ascending=False)[:20]
top_r = rating_data.join(top_users, rsuffix='_r', how='inner', on='user_id')

g = rating_data.groupby('MAL_ID')['rating'].count()
top_animes = g.dropna().sort_values(ascending=False)[:20]
top_r = top_r.join(top_animes, rsuffix='_r', how='inner', on='MAL_ID')

pivot = pd.crosstab(top_r.user_id, top_r.MAL_ID, top_r.rating, aggfunc=np.sum)
pivot.fillna(0, inplace=True)
pivot.head()

MAL_ID,226,1535,1575,2904,4224,5081,5114,6547,8074,9253,9989,10620,11111,11757,14741,15809,16498,19815,20507,30276
user_id,,,,,,,,,,,,,,,,,,,,
10255,2573.0,2672.0,2671.0,2576.0,2659.0,2629.0,2553.0,2776.0,2625.0,2638.0,2544.0,2536.0,2594.0,0.0,2542.0,2599.0,2727.0,2673.0,2547.0,2670.0
64807,2573.0,2672.0,2671.0,2576.0,2659.0,2629.0,2553.0,2776.0,2625.0,2638.0,2544.0,2536.0,2594.0,2730.0,2542.0,2599.0,2727.0,2673.0,2547.0,2670.0
68042,2573.0,2672.0,2671.0,2576.0,2659.0,2629.0,2553.0,2776.0,2625.0,2638.0,2544.0,2536.0,2594.0,2730.0,2542.0,2599.0,2727.0,2673.0,2547.0,2670.0
85106,2573.0,2672.0,2671.0,2576.0,2659.0,2629.0,2553.0,2776.0,2625.0,2638.0,2544.0,2536.0,2594.0,2730.0,2542.0,2599.0,2727.0,2673.0,2547.0,2670.0
122341,2573.0,2672.0,2671.0,2576.0,2659.0,2629.0,2553.0,2776.0,2625.0,2638.0,2544.0,2536.0,2594.0,2730.0,2542.0,2599.0,2727.0,2673.0,2547.0,2670.0


In [152]:
piviot_table = rating_data.pivot_table(index="Name",columns="user_id", values="rating").fillna(0)
piviot_table.head()

user_id,478,781,853,890,912,943,1177,1231,1393,1397,...,352366,352478,352660,352669,352761,352832,352835,352922,352924,352930
Name,,,,,,,,,,,,,,,,,,,,,
"""0""",0.0,0.0,0.0,307.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,307.0,0.0,0.0
"""Aesop"" no Ohanashi yori: Ushi to Kaeru, Yokubatta Inu",0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""Bungaku Shoujo"" Kyou no Oyatsu: Hatsukoi",0.0,0.0,0.0,0.0,0.0,0.0,0.0,922.0,0.0,922.0,...,0.0,922.0,0.0,0.0,922.0,0.0,0.0,0.0,0.0,922.0
"""Bungaku Shoujo"" Memoire",0.0,0.0,0.0,0.0,0.0,0.0,0.0,982.0,0.0,982.0,...,0.0,0.0,982.0,982.0,982.0,0.0,0.0,982.0,0.0,982.0
"""Bungaku Shoujo"" Movie",0.0,0.0,0.0,0.0,0.0,0.0,0.0,1196.0,0.0,1196.0,...,0.0,1196.0,1196.0,1196.0,1196.0,0.0,0.0,0.0,1196.0,1196.0


In [153]:
from scipy.sparse import csr_matrix
piviot_table_matrix = csr_matrix(piviot_table.values)

In [154]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(metric="cosine", algorithm="brute")
model.fit(piviot_table_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [155]:
def predict(anime_name_input, n_neighbors):
    query = piviot_table.loc[anime_name_input, :].values.reshape(1, -1)

    distance, suggestions = model.kneighbors(query, n_neighbors)
    recommendation = {}
    for i in range(0, len(distance.flatten())):
        if i == 0:
            print('Recommendations for', anime_name_input)
        else:
            print('{0}: {1}, with distance of {2}:'.format(i, piviot_table.index[suggestions.flatten()[i]], distance.flatten()[i]))
            recommendation[piviot_table.index[suggestions.flatten()[i]]] = distance.flatten()[i]

    return recommendation

In [156]:
top_animes_count = 1000
animes_popular = anime_df.sort_values('Completed', ascending=False)[0:top_animes_count]
recommendation_for_all = animes_popular[:100].sort_values(by='Score', ascending=False)[:10]

In [157]:
def recommend_for_user(user_id_input):
    '''
    Take user_id to get top-rated anime by the user, then for each anime make prediction
    return the highest 10 correlation anime
    :param user_id_input: user id
    :return: dataframe of recommended anime
    '''
    top_users = rating_popular_anime.user_id.unique()
    recommendation_df = pd.DataFrame(columns=['MAL_ID'])
    users_groups = rating_popular_anime.groupby('user_id')
    if user_id_input in top_users:
        user_fav_anime = users_groups.get_group(user_id_input).sort_values(by='rating', ascending=False)[:5]['MAL_ID'].tolist()
        for user_fav_MAL_ID in user_fav_anime:
            anime_name=anime_df[anime_df['MAL_ID']==user_fav_MAL_ID]['Name'].values[0]
            r = predict(anime_name, 5)
            r_df = anime_df[(anime_df['Name'].isin(r.keys()))]
            distances = []
            for index, row in r_df.iterrows():
                distances.append(r[row['Name']])
            r_df.insert(2, "Distance", distances, True)
            recommendation_df = recommendation_df.append(r_df, ignore_index=True)

        recommendation_df = recommendation_df.sort_values(by='Distance', ascending=True).drop_duplicates('MAL_ID')
        return recommendation_df[:10]
    else:
        return recommendation_for_all

In [158]:
recommend_for_user(478)

Recommendations for Psycho-Pass
1: Psycho-Pass 2, with distance of 0.07694537566275172:
2: Shingeki no Kyojin, with distance of 0.09494705789882452:
3: Sword Art Online, with distance of 0.1031526615439039:
4: Steins;Gate, with distance of 0.10411498451345291:
Recommendations for Boku dake ga Inai Machi
1: One Punch Man, with distance of 0.0904172253335338:
2: Shingeki no Kyojin, with distance of 0.09346268637738897:
3: No Game No Life, with distance of 0.09634354597216832:
4: Angel Beats!, with distance of 0.09770313900570327:
Recommendations for Durarara!!x2 Ketsu
1: Durarara!!x2 Ten, with distance of 0.04184940109145896:
2: Durarara!!x2 Shou, with distance of 0.07319641970309909:
3: Durarara!!, with distance of 0.16729613872113147:
4: Boku dake ga Inai Machi, with distance of 0.22182183687714074:
Recommendations for Hikaru no Go
1: Hikaru no Go: Hokuto Hai e no Michi, with distance of 0.2435306674003449:
2: Hikaru no Go: New Year Special, with distance of 0.40588897917019173:
3: Ten

,MAL_ID,Name,Distance,Score,Genres,Type,Episodes,Premiered,Studios,Source,Rating,Members,Completed
10,27831,Durarara!!x2 Ten,0.041849,8.01,"Action, Mystery, Supernatural",TV,12,Summer 2015,Shuka,Light novel,R - 17+ (violence & profanity),312429.0,216764.0
9,23199,Durarara!!x2 Shou,0.073196,8.03,"Action, Mystery, Supernatural",TV,12,Winter 2015,Shuka,Light novel,R - 17+ (violence & profanity),388580.0,267808.0
3,23281,Psycho-Pass 2,0.076945,7.42,"Action, Sci-Fi, Police, Psychological",TV,11,Fall 2014,Tatsunoko Production,Original,R - 17+ (violence & profanity),508172.0,352491.0
7,30276,One Punch Man,0.090417,8.57,"Action, Sci-Fi, Comedy, Parody, Super Power, Supernatural",TV,12,Fall 2015,Madhouse,Web manga,R - 17+ (violence & profanity),2123866.0,1841220.0
19,16498,Shingeki no Kyojin,0.091173,8.48,"Action, Military, Mystery, Super Power, Drama, Fantasy, Shounen",TV,25,Spring 2013,Wit Studio,Manga,R - 17+ (violence & profanity),2531397.0,2182587.0
18,6547,Angel Beats!,0.092093,8.15,"Action, Comedy, Drama, School, Supernatural",TV,13,Spring 2010,P.A. Works,Original,PG-13 - Teens 13 or older,1591773.0,1229098.0
17,1575,Code Geass: Hangyaku no Lelouch,0.094715,8.72,"Action, Military, Sci-Fi, Super Power, Drama, Mecha, School",TV,25,Fall 2006,Sunrise,Original,R - 17+ (violence & profanity),1583882.0,1209288.0
16,1535,Death Note,0.094885,8.63,"Mystery, Police, Psychological, Supernatural, Thriller, Shounen",TV,37,Fall 2006,Madhouse,Manga,R - 17+ (violence & profanity),2589552.0,2146116.0
6,19815,No Game No Life,0.096344,8.2,"Game, Adventure, Comedy, Supernatural, Ecchi, Fantasy",TV,12,Spring 2014,Madhouse,Light novel,PG-13 - Teens 13 or older,1751054.0,1426896.0
1,11757,Sword Art Online,0.103153,7.25,"Action, Game, Adventure, Romance, Fantasy",TV,25,Summer 2012,A-1 Pictures,Light novel,PG-13 - Teens 13 or older,2214395.0,1907261.0
